In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import random

In [2]:
input_path = 'sto/'
kg = input_path + 'sto-enriched-relatedTo.nt'
dataset = pd.read_csv(kg, delimiter='---', header=None)
dataset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,0
0,<http://www.w3.org/1999/02/22-rdf-syntax-ns#va...
1,<http://www.w3.org/2001/XMLSchema#gYear> <http...
2,<http://www.w3.org/2001/XMLSchema#date> <http:...
3,<http://purl.org/dc/elements/1.1/relation> <ht...
4,<http://purl.org/dc/elements/1.1/description> ...


In [3]:
g = Graph()
g.parse("sto/sto-enriched-relatedTo.nt", format="nt")

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
    } limit 1000""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

## Select all standard triples given a specific property

In [4]:
count = 0
#prop = 'hasClassification'
prop = 'relatedTo'
list_pd = pd.DataFrame(columns=['s', 'p', 'o'])
for i in range(entities.shape[0]):
    standard = entities.s[i]
    qres = g.query(
        """PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX sto: <https://w3id.org/i40/sto#>
            select distinct ?o where {
                <""" +standard+ """> sto:""" +prop+ """ ?o .
            }""")
    lst = pd.DataFrame(columns=['s', 'p', 'o'])
    obj = []
    for row in qres:
        obj.append('<'+str("%s" %row)+'>')
    
    if len(obj)>0:
        lst.o=obj
        lst.s='<'+standard+'>'
        lst.p='<https://w3id.org/i40/sto#'+prop+'>'
        list_pd = pd.concat([list_pd, lst], ignore_index=True)
    else:
        count+=1

#list_pd = list_pd.drop_duplicates().reset_index(drop=True)

## Create the test set [Select 30% of triple randomly] 

In [20]:
n_select= round(3*list_pd.shape[0]/10)
print(n_select)
index = list_pd.index.tolist()
#sampling = random.choices(index, k=n_select)
sampling = random.sample(index, k=n_select)

880


In [22]:
test_set=list_pd.iloc[sampling]
duplicateRowsDF = test_set[test_set.duplicated()]
test_set = test_set.drop_duplicates().reset_index(drop=True)
print(test_set.shape[0])
duplicateRowsDF

880


,s,p,o


In [ ]:
#test_set.sort_values(by='s', ascending=False)

In [23]:
test_set['dot'] = '.'

In [24]:
test_set_nt=pd.DataFrame(test_set['s'] + ' '+ test_set['p']+ ' '+ test_set['o']+ ' '+ test_set['dot'])
test_set_nt.head(20)

,0
0,<https://w3id.org/i40/sto#SysML> <https://w3id...
1,<https://w3id.org/i40/sto#IEC_81714> <https://...
2,<https://w3id.org/i40/sto#RFC_7540> <https://w...
3,<https://w3id.org/i40/sto#ISO_ASTM_52915> <htt...
4,<https://w3id.org/i40/sto#RFC_7519> <https://w...
5,<https://w3id.org/i40/sto#DIN_SPEC_27070> <htt...
6,<https://w3id.org/i40/sto#IEC_24760> <https://...
7,<https://w3id.org/i40/sto#IEC_61512> <https://...
8,<https://w3id.org/i40/sto#ISO_1302> <https://w...
9,<https://w3id.org/i40/sto#ISO_29002> <https://...


## Create training test [difference bettwen dataset and test set]

In [25]:
training_set = dataset[~dataset.apply(tuple,1).isin(test_set_nt.apply(tuple,1))]
training_set.shape[0]

22558

In [26]:
dataset.shape[0]-training_set.shape[0]

880

## Save training and test set

In [32]:
training_set.to_csv(input_path+'training_set_relatedTo.nt', index=None, header=None)
test_set_nt.to_csv(input_path+'test_set_relatedTo.nt', index=None, header=None)

## Cheking the training set created

In [40]:
g = Graph()
g.parse("sto/training_set_relatedTo.nt", format="nt")
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
    } limit 1000""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [41]:
entities

,s
0,https://w3id.org/i40/sto#ISA_62443
1,https://w3id.org/i40/sto#ISA-95
2,https://w3id.org/i40/sto#ISO_29190
3,https://w3id.org/i40/sto#IEC_61158
4,https://w3id.org/i40/sto#IEC_61310_P1_E2
5,https://w3id.org/i40/sto#IEC_61804_P4
6,https://w3id.org/i40/sto#ISO_20140_P1
7,https://w3id.org/i40/sto#IEC_61334_P5_S1_E2
8,https://w3id.org/i40/sto#RFC_7644
9,https://w3id.org/i40/sto#IEC_60870_P5_S104_E2_1
